Студент Жилкин Федор, 344
# Домашняя работа 4: "Численное решение интегрального уравнения Фредгольма 2-го рода."
**Задания:**

1. Метод механических квадратур  
2. Метод замены ядра на вырожденное

$u(x) + 0.5\int\limits_0^1 e^{xy}u(y)dy = x + 0.5$

In [62]:
import numpy as np
import pandas as pd 
import sympy

In [108]:
# Функция f(x) справа
def f(x):
    return x + 0.5 

In [109]:
# Ядро
def K(x, y):
    return np.exp(x*y)

## Метод механических квадратур

In [159]:
# Метод механических квадратур
def mechanical_quadrature(n, X):
     
    """
    Решение интегрального уравнения методом механических квадратур
    n -- кол-во точек разбиения отрезка
    X -- точка
    default: up, down = 1, 0
    """
    
    # Задаем равномерную сетку
    up, down = 1, 0
    h = (up - down)/n
    x = [(down + i*h) for i in range(0, n)]
    
    A = np.zeros((n, n))
    B = np.zeros(n)
    
    # Веса составной формулы средних прямоугольников
    weights = np.ones(n)*h 
    weights[0] /= 2
    weights[n-1] /= 2
    
    for i in range(0, n):
        for j in range(0, n):
            if j == i:
                beta = 1
            else:
                beta = 0
            A[i][j] = beta - weights[j]*K(x[i], x[j])
        B[i] = f(x[i])
    
    # Нашли искомый вектор Z
    Z = np.linalg.solve(A, B)
    
    # Решение уравнения
    sum_ = sum([weights[i]*K(X, x[i])*Z[i] for i in range(0, n)])
    u = -0.5*sum_ + f(X)
    return u

In [162]:
mechanical_quadrature(1200,1)

4.098714964616895

In [194]:
x = sympy.Symbol('x')
f = sympy.integrate(x/(x**2+2*x+1), x)

In [213]:
f

log(x + 1) + 1/(x + 1)

In [215]:
d = sympy.lambdify(x, f, 'numpy')

In [218]:
d(0)

1.0

Ядро $e^{xy}$ заменяем на вырожденное ядро: $e^{xy} = \sum\limits^{n}_{k=0}{\frac{(xy)^{k}}{k!}} = \sum\limits^{n}_{k=0}x^k \frac{y^k}{k!} = \sum\limits^{n}_{k=0}a_k(x)b_k(y)$

Отсюда $a_k(x) = x^k$

In [ ]:
def degenerate_kernel(n, X):
    
    """
    Решение интегрального уравнения методом замены ядра на вырожденное
    n -- кол-во точек разбиения отрезка
    X -- точка
    default: up, down = 1, 0
    """
    
    # Задаем равномерную сетку
    up, down = 1, 0
    h = (up - down)/n
    x = [(down + i*h) for i in range(0, n)]
    
    A = np.zeros((n, n))
    B = np.zeros(n)
    
    
    